# Converting Notebook Code to a Function

The `# mlrun: ...` annotations are used to identify the code that needs to be converted into an MLRun function.
These annotations provide non-intrusive hints as to what parts of your notebook should be considered as the code of the function.

Everything before the `# mlrun: start-code` annotation and after the `# mlrun: end-code` annotation is ignored, and only code between these two annotations is converted.
Make sure to include imports and anything required for the function in the included code.

In [1]:
# mlrun: start-code

def sub_handler():
    return "hello world"

The `# mlrun: ignore` annotation enables you to exclude the cell from the function code.

In [2]:
# mlrun: ignore

# the handler in the code section below will not call this sub_handler
def sub_handler():
    return "I will be ignored!"

In [3]:
def handler(context, event):
    return sub_handler()

# mlrun: end-code

Let's convert the function with `mlrun.code_to_function` and run the handler, notice the returned value under `results`.

In [4]:
from mlrun import code_to_function

some_function = code_to_function('some-function-name', kind='job', code_output='.')
some_function.run(name='some-function-name', handler='handler', local=True)

> 2021-11-01 07:30:18,941 [info] starting run some-function-name uid=6c655d8595334b75b9233b4a1f20f0a7 DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...1f20f0a7,0,Nov 01 07:30:19,completed,some-function-name,v3io_user=adminkind=owner=adminhost=jupyter-8459699595-z544v,,,return=hello world,


> 2021-11-01 07:30:19,233 [info] run executed, status=completed


### Named annotations
The `# mlrun: start-code` and `# mlrun: end-code` annotations can be used to convert different code sections to different MLRun functions in the same notebook.
To do so add the name of the MLRun function to the end of the annotation similarly to the code block below.
> **Note:**<br> Make sure to use the name given to `code_to_function` as a parameter (name='my-fucntion-name' in the example below) and to mark all relevant `start-code` and `end-code` annotations, if none of them are marked with the function's name, the annotations without any name will be used.

In [5]:
# mlrun: start-code my-fucntion-name

def handler(context, event):
    return "hello from my-fucntion"

# mlrun: end-code my-fucntion-name

Let's convert the function and run the handler, notice the handler that is being used and the change in the returned value under `results`.

In [6]:
my_fucntion = code_to_function('my-fucntion-name', kind='job')
my_fucntion.run(name='my-fucntion-name', handler='handler', local=True)

> 2021-11-01 07:30:28,168 [info] starting run my-fucntion-name uid=d6c2dd3af0e04c36b0d1bf1e618104ff DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...618104ff,0,Nov 01 07:30:28,completed,my-fucntion-name,v3io_user=adminkind=owner=adminhost=jupyter-8459699595-z544v,,,return=hello from my-fucntion,


> 2021-11-01 07:30:28,424 [info] run executed, status=completed


### Multi section function
You can use the `# mlrun: start-code` and `# mlrun: end-code` annotations multiple times in a notebook, the whole notebook will be scanned.
The annotations can be named like the following example, and they can be nameless (if you choose nameless remember all nameless annotations in the notebook will be used).

In [7]:
# mlrun: start-code multi-section-function-name

function_name = "multi-section-function-name"

# mlrun: end-code multi-section-function-name

Any code between those sections will not be included:

In [8]:
function_name = "I will be ignored!"

In [9]:
# mlrun: start-code multi-section-function-name

In [10]:
def handler(context, event):
    return f"hello from {function_name}"

In [11]:
# mlrun: end-code multi-section-function-name

In [12]:
my_multi_section_fucntion = code_to_function('multi-section-function-name', kind='job')
my_multi_section_fucntion.run(name='multi-section-function-name', handler='handler', local=True)

> 2021-11-01 07:30:37,025 [info] starting run multi-section-function-name uid=b92ded3bed0e4cd9bc048abbb033201c DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...b033201c,0,Nov 01 07:30:37,completed,multi-section-function-name,v3io_user=adminkind=owner=adminhost=jupyter-8459699595-z544v,,,return=hello from multi-section-function-name,


> 2021-11-01 07:30:37,287 [info] run executed, status=completed


### Annotation's position in code cell

`# mlrun: start-code` and `# mlrun: end-code` annotations are relative to their positions inside the code block, notice how assignments to `function_name` below `# mlrun: end-code` doesn't override the assignment between the annotations in the function's context.

In [13]:
# mlrun: start-code part-cell-fucntion

def handler(context, event):
    return f"hello from {function_name}"

function_name = "part-cell-fucntion"

# mlrun: end-code part-cell-fucntion

function_name = "I will be ignored"

In [14]:
my_multi_section_fucntion = code_to_function('part-cell-fucntion', kind='job')
my_multi_section_fucntion.run(name='part-cell-fucntion', handler='handler', local=True)

> 2021-11-01 07:30:46,146 [info] starting run part-cell-fucntion uid=99b37df30fb94458b02132cf703f2ad6 DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...703f2ad6,0,Nov 01 07:30:46,completed,part-cell-fucntion,v3io_user=adminkind=owner=adminhost=jupyter-8459699595-z544v,,,return=hello from part-cell-fucntion,


> 2021-11-01 07:30:46,398 [info] run executed, status=completed


## Notes
- Make sure not to have consecutive `# mlrun: start-code` without `# mlrun: end-code` in between them and vice versa.
- Only 1 MLRun function can use nameless annotations per notebook.
- Do not use multiple `# mlrun: start-code` nor multiple `# mlrun: end-code` annotations in a single code cell, only the 1st appearance of each will be used.
- You may use a single annotation:
    1. using `# mlrun: start-code` alone will include code blocks from the annotation to the end of the notebook.
    2. using `# mlrun: end-code` alone will include code blocks from the beginning of the notebook to the annotation.